# 📝 Exercise M3.01

The goal is to write an exhaustive search to find the best parameters
combination maximizing the model generalization performance.

Here we use a small subset of the Adult Census dataset to make the code
faster to execute. Once your code works on the small subset, try to
change `train_size` to a larger value (e.g. 0.8 for 80% instead of
20%).

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

adult_census = pd.read_csv("../datasets/adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.2, random_state=42)

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)
preprocessor = ColumnTransformer(
    [('cat_preprocessor', categorical_preprocessor,
      selector(dtype_include=object))],
    remainder='passthrough', sparse_threshold=0)

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=42))
])


Use the previously defined model (called `model`) and using two nested `for`
loops, make a search of the best combinations of the `learning_rate` and
`max_leaf_nodes` parameters. In this regard, you will need to train and test
the model by setting the parameters. The evaluation of the model should be
performed using `cross_val_score` on the training set. We will use the
following parameters search:
- `learning_rate` for the values 0.01, 0.1, 1 and 10. This parameter controls
  the ability of a new tree to correct the error of the previous sequence of
  trees
- `max_leaf_nodes` for the values 3, 10, 30. This parameter controls the
  depth of each tree.

In [7]:
model.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                     transformers=[('cat_preprocessor',
                                    OrdinalEncoder(handle_unknown='use_encoded_value',
                                                   unknown_value=-1),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x7f26ce377a90>)])),
  ('classifier', HistGradientBoostingClassifier(random_state=42))],
 'verbose': False,
 'preprocessor': ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                   transformers=[('cat_preprocessor',
                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                 unknown_value=-1),
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f26ce377a90>)]),
 'classifier': HistGradientBoostingClassifier(random_st

In [9]:
from sklearn.model_selection import cross_val_score

learning_rates = [0.01, 0.1, 1,10]
max_leaf_nodes = [3,10,30]

for learning_rate in learning_rates:
    for max_leaf_node in max_leaf_nodes:
        model.set_params(classifier__learning_rate=learning_rate, classifier__max_leaf_nodes=max_leaf_node)
        results = cross_val_score(model, data_train, target_train)
        print(f"max leaf node:{max_leaf_node}, learning rate:{learning_rate}\n{results.mean():.3f} +/-{results.std():.3f}\n")

max leaf node:3, learning rate:0.01
0.790 +/-0.004

max leaf node:10, learning rate:0.01
0.814 +/-0.002

max leaf node:30, learning rate:0.01
0.842 +/-0.007

max leaf node:3, learning rate:0.1
0.849 +/-0.004

max leaf node:10, learning rate:0.1
0.863 +/-0.005

max leaf node:30, learning rate:0.1
0.861 +/-0.006

max leaf node:3, learning rate:1
0.854 +/-0.006

max leaf node:10, learning rate:1
0.837 +/-0.008

max leaf node:30, learning rate:1
0.819 +/-0.013

max leaf node:3, learning rate:10
0.288 +/-0.008

max leaf node:10, learning rate:10
0.646 +/-0.145

max leaf node:30, learning rate:10
0.534 +/-0.191




Now use the test set to score the model using the best parameters
that we found using cross-validation in the training set.

max leaf node of 10 and learning rate of 0.1 gives highest accuracy

In [12]:
model.set_params(classifier__learning_rate=0.1, classifier__max_leaf_nodes=10)
model.fit(data_train, target_train)
model.score(data_test, target_test)
print(f"max leaf node:{10}, learning rate:{0.1}\n{results.mean():.3f} +/-{results.std():.3f}\n")

max leaf node:10, learning rate:0.1
0.872 +/-0.004

